# AI Agent for Data Science using LIDA

## **Overview**

In this notebook, we'll create an AI agent capable of performing complex data science tasks using LIDA. The agent will:

1. **Generate Analysis**: Summarize and visualize data based on user queries.
2. **Visual Analysis**: Analyze generated charts/images to provide detailed insights.

We'll leverage various tools and services, including Azure OpenAI, LIDA, and other Python libraries.

---

## **Table of Contents**

1. [Setup and Installation](#Setup-and-Installation)
2. [Importing Libraries](#Importing-Libraries)
3. [Environment Configuration](#Environment-Configuration)
4. [Helper Functions](#Helper-Functions)
    - [a. Fetch Current Date and Time](#a.-Fetch-Current-Date-and-Time)
    - [b. Generate Analysis](#b.-Generate-Analysis)
    - [c. Visual Analysis](#c.-Visual-Analysis)
5. [AI Agent Setup](#AI-Agent-Setup)
    - [a. Define the AI Agent Tool](#a.-Define-the-AI-Agent-Tool)
    - [b. Response Formatting Function](#b.-Response-Formatting-Function)
6. [Using the AI Agent](#Using-the-AI-Agent)
7. [Conclusion](#Conclusion)

---

## **1. Setup and Installation**

First, we'll install all the necessary Python packages required for our AI agent. Ensure you have the latest versions of `pip` and `conda` (if using).

# Install required packages
!pip install python-dotenv openai azure-identity azure-ai-projects azure-monitor-opentelemetry lida llm llmx 

> **Note**: Some packages like `lida`, `llm`, `llmx`, and `lida_tool` might be custom or proprietary. Ensure they are available in your environment or install them from the appropriate sources.

---

## **2. Importing Libraries**

Import all the necessary libraries and modules. This includes standard Python libraries as well as specialized modules for AI and data analysis.

In [1]:
import json
import time
from lida import Manager, TextGenerationConfig
from llmx import llm, TextGenerationConfig
import os
from typing import Set, Callable, Dict, Any, List, Optional
from pathlib import Path
from dotenv import load_dotenv
import datetime
import base64
from openai import AzureOpenAI

# Azure and telemetry imports
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import (
    AsyncFunctionTool, 
    RequiredFunctionToolCall, 
    SubmitToolOutputsAction, 
    ToolOutput, 
    AsyncToolSet,
    CodeInterpreterTool,
    BingGroundingTool
)
from azure.ai.projects.telemetry.agents import AIAgentsInstrumentor
from azure.identity.aio import DefaultAzureCredential
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace
import pandas as pd

---

## **3. Environment Configuration**

Set up environment variables using a `.env` file. This file should contain sensitive information like API keys and connection strings.

1. **Create a `.env` File**: Ensure you have a `deploy.env` file in your working directory with the following variables:

    ```
    AZURE_OPENAI_BASE=your_azure_openai_base_endpoint
    AZURE_OPENAI_API_KEY=your_azure_openai_api_key
    PROJECT_CONNECTION_STRING=your_project_connection_string
    ```

2. **Load Environment Variables**:

In [2]:
# Load environment variables from deploy.env
load_dotenv('.env')

True

---

## **4. Helper Functions**

Define helper functions that will be used by the AI agent for data analysis and visualization.

#### **a. Fetch Current Date and Time**

Gets the current date and time, formatted as specified.

In [3]:
def fetch_current_datetime(format: Optional[str] = None) -> str:
    """
    Get the current time as a JSON string, optionally formatted.

    :param format (Optional[str]): The format in which to return the current time. Defaults to None.
    :return: The current time in JSON format.
    :rtype: str
    """
    current_time = datetime.datetime.now()

    # Use the provided format if available, else use a default format
    if format:
        time_format = format
    else:
        time_format = "%Y-%m-%d %H:%M:%S"

    time_json = json.dumps({"current_time": current_time.strftime(time_format)})
    return time_json

## Test the LIDA tool

LIDA is a tool for performing data analysis using Generative AI, [github.com/microsoft/lida](https://github.com/microsoft/lida)

Make sure you have installed LIDA package

``` pip install lida ```


In [4]:
data_input = './data/financial_sample.xlsx'
question = "What was the profit for Carreterra in Germany for Dec?"
output_folder = './output'
model_deployment = os.environ.get("CHAT_MODEL_DEPLOYMENT_NAME")

# Initialize LLM with Azure OpenAI
text_gen = llm(
    provider="openai",
    api_type="azure",
    azure_endpoint=os.environ.get("CHAT_MODEL_ENDPOINT"),
    api_key=os.environ.get("CHAT_MODEL_API_KEY"),
    api_version="2023-07-01-preview",
)
    # Load the Excel File into a DataFrame
df = pd.read_excel(data_input)
print(f"Workbook '{data_input}' successfully loaded.")

lida = Manager(text_gen=text_gen)

# Configure text generation
textgen_config = TextGenerationConfig(n=1, temperature=0.5, model=model_deployment, use_cache=False)

# Summarize the input data
summary = lida.summarize(df, summary_method="default", textgen_config=textgen_config)  

# Visualize the summary based on the question
charts = lida.visualize(summary=summary, goal=question, textgen_config=textgen_config)  

if len(charts) > 0:
    chart = charts[0]
    code = chart.code

    # Create a timestamp for saving files
    run_timestamp = str(int(time.time()))
    
    #create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Save summary to a Python file
    #with open(f'{output_folder}/summary_{run_timestamp}.py', 'w') as f:
    #    f.write(str(summary))
    
    # Save generated code to a Python file
    with open(f'{output_folder}/code_{run_timestamp}.py', 'w') as f:
        f.write(code)
    
    # Save the chart image
    chart.savefig(f'{output_folder}/chart_{run_timestamp}.png')


Workbook './data/financial_sample.xlsx' successfully loaded.


<string>:21: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.



#### **b. Generate Analysis**

Generates a summary and visual charts based on the provided question and data.

In [5]:
async def generate_analysis(question: str,model_deployment:str, data_input: str, output_folder:str) -> str:
    """
    Generate a summary and visualize the data based on the question.

    :param question (str): The analysis question.
    :param data_input (str): The data to analyze.
    :return: Path to the saved chart image or an error message.
    :rtype: str
    """
    # Initialize LLM with Azure OpenAI
    text_gen = llm(
    provider="openai",
    api_type="azure",
    azure_endpoint=os.environ.get("CHAT_MODEL_ENDPOINT"),
    api_key=os.environ.get("CHAT_MODEL_API_KEY"),
    api_version="2023-07-01-preview",
        )
        # Load the Excel File into a DataFrame
    df = pd.read_excel(data_input)
    print(f"Workbook '{data_input}' successfully loaded.")

    lida = Manager(text_gen=text_gen)

    # Configure text generation
    textgen_config = TextGenerationConfig(n=1, temperature=0.5, model=model_deployment, use_cache=False)

    # Summarize the input data
    summary = lida.summarize(df, summary_method="default", textgen_config=textgen_config)  

    # Visualize the summary based on the question
    charts = lida.visualize(summary=summary, goal=question, textgen_config=textgen_config)  

    if len(charts) > 0:
        chart = charts[0]
        code = chart.code

        # Create a timestamp for saving files
        run_timestamp = str(int(time.time()))
        
        #create output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)
        
        # Save summary to a Python file
        #with open(f'{output_folder}/summary_{run_timestamp}.py', 'w') as f:
        #    f.write(str(summary))
        
        # Save generated code to a Python file
        with open(f'{output_folder}/code_{run_timestamp}.py', 'w') as f:
            f.write(code)
        
        # Save the chart image
        chart.savefig(f'{output_folder}/chart_{run_timestamp}.png')

        return f'{output_folder}/chart_{run_timestamp}.png'
    else:
        field_list = str(summary['field_names'])
        return f"Unable to visualize question, please try again with these fields: {field_list}"

#### **c. Visual Analysis**

Analyzes the generated chart image using vision capability of MLLM to provide detailed insights.

In [6]:
async def visual_analysis(question: str, img_path: str, deployment_model:str) -> str:
    """
    Analyze the chart image and provide detailed insights.

    :param question (str): The analysis question.
    :param img_path (str): Path to the chart image.
    :return: Detailed analysis in JSON format.
    :rtype: str
    """
    # Initialize Azure OpenAI client
    client = AzureOpenAI(  
        azure_endpoint=os.environ.get("CHAT_MODEL_ENDPOINT"),
        api_key=os.environ.get("CHAT_MODEL_API_KEY"),
        api_version="2024-05-01-preview",  
    )  

    # Read and encode the image
    with open(img_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('ascii')

    # Define the chat prompt
    chat_prompt = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "You are an AI assistant that helps people find information."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"Please analyze the image and provide a detailed analysis of the chart for this question: {question}"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_image}"
                    }
                }
            ]
        }
    ] 
    
    # Include speech result if speech is enabled  
    messages = chat_prompt  
        
    # Generate the completion  
    completion = client.chat.completions.create(  
        model=deployment_model,
        messages=messages,
        max_tokens=800,  
        temperature=0,  
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False
    )

    return completion.to_json()  
        

# Statically defined user functions for fast reference with send_email as async but the rest as sync
user_async_function_tools: Set[Callable[..., Any]] = {
    generate_analysis,
    visual_analysis
}

---

## **5. AI Agent Setup**

Configure and initialize the AI agent using Azure services and LIDA tools.

In [7]:
# Define user asynchronous function tools
user_async_function_tools: Set[Callable[..., Any]] = {
    generate_analysis,
    visual_analysis
}

# Initialize tracer for telemetry
tracer = trace.get_tracer(__name__)

#### **a. Define the AI Agent Tool**

Create a tool that the AI agent will use to perform data analysis tasks.

In [ ]:
async def analysis_agent(question: str, data_input: str, output_folder:str, model_deployment: str) -> str:
    """
    Main function to handle AI agent operations.

    :param question (str): The analysis question.
    :param data_input (str): The data input for analysis.
    :param model_deployment (str): The model deployment configuration.
    :return: AI agent's response.
    :rtype: str
    """
    async with DefaultAzureCredential() as creds:
        async with AIProjectClient.from_connection_string(
            credential=creds, conn_str=os.environ["AIPROJECT_CONNECTION_STRING"],
        ) as project_client:
            
            # Configure Azure Monitor for telemetry
            application_insights_connection_string = await project_client.telemetry.get_connection_string()
            configure_azure_monitor(connection_string=application_insights_connection_string)
            
            # Initialize assistant functions
            functions = AsyncFunctionTool(functions=user_async_function_tools)
            code_interpreter = CodeInterpreterTool()

            # Setup toolset
            toolset = AsyncToolSet()
            toolset.add(functions)
            # Uncomment the next line if you want to add the code interpreter
            # toolset.add(code_interpreter)

            agent_name = "data-science-assistant3"

            # Check if the agent already exists
            agents = await project_client.agents.list_agents()
            agent = next((a for a in agents.data if a.name == agent_name), None)

            if agent is None:
                # Create a new agent if not found
                agent = await project_client.agents.create_agent(
                    model=model_deployment,
                    name=agent_name,
                    instructions=(
                        'You are a data scientist with access to a tool called generate_analysis '
                        'which can perform analysis and save results for you. Use the data_input provided. '
                        'Use the generate_analysis tool to answer the question, then use the visual_analysis '
                        'to process the image. The answer should be no greater than 1000 characters in length.'
                    ),
                    tools=functions.definitions #+ code_interpreter.definitions
                )
                print(f"Created agent, agent ID: {agent.id}")
            else:
                print(f"Found existing agent: {agent.id}")

            # Create a thread for communication
            thread = await project_client.agents.create_thread()
            print(f"Created thread, ID: {thread.id}")
            
            # Send a message to the agent
            message = await project_client.agents.create_message(
                thread_id=thread.id, 
                role="user", 
                content=f"Current date is {datetime.datetime.now().strftime('%Y-%m-%d')}.model_deployment:{model_deployment}, {question},output_folder:{output_folder}, data_input:{data_input}"
            )
            print(f"Created message, ID: {message.id}")

            # Process the agent run with the provided tools
            run = await project_client.agents.create_and_process_run(
                thread_id=thread.id, 
                agent_id=agent.id, 
                toolset=toolset
            )
            print(f"Run finished with status: {run.status}")

            if run.status == "failed":
                print(f"Run failed: {run.last_error}")

            print(f"Run completed with status: {run.status}")

            # Fetch and log all messages from the thread
            messages = await project_client.agents.list_messages(thread_id=thread.id)
            print(f"Messages: {messages}")

            # Save any generated files (e.g., images)
            for file_path_annotation in messages.file_path_annotations:
                print(f"File Paths:")
                print(f"Type: {file_path_annotation.type}")
                print(f"Text: {file_path_annotation.text}")
                print(f"File ID: {file_path_annotation.file_path.file_id}")
                print(f"Start Index: {file_path_annotation.start_index}")
                print(f"End Index: {file_path_annotation.end_index}")
                file_name = Path(file_path_annotation.text).name
                await project_client.agents.save_file(
                    file_id=file_path_annotation.file_path.file_id, 
                    file_name=file_name
                )
                print(f"Saved image file to: {Path.cwd() / file_name}")

            # Get the last message from the conversation
            last_message = messages.text_messages[0].text
            response = last_message
            return response

---

## **6. Using the AI Agent**

With the AI agent set up, you can now interact with it by providing questions and data inputs for analysis.

In [12]:
# Define the question and data input
question = "what were Profits for Carreterra in Germany in December?"
data_input = "./data/financial_sample.xlsx" #this can also be a local file

# Define the model deployment (ensure this matches your Azure OpenAI deployment)
model_deployment = "gpt-4o"
output_folder = "output"

# Run the AI agent
response = await analysis_agent(question, data_input,output_folder, model_deployment)

# Display the response
print("AI Agent Response:")
print(response['value'])

Found existing agent: asst_TlTSxQoGlrUv1uAoBUWPwlhU
Created thread, ID: thread_pHIzM7sGYEQAsLpN1xPiKJls
Created message, ID: msg_osVEH9DLqMdmp9sHefFicdbr
Workbook './data/financial_sample.xlsx' successfully loaded.


<string>:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.



Run finished with status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_G7LioQ1yUYdlB6zZuz9qx7FX', 'object': 'thread.message', 'created_at': 1743095743, 'assistant_id': 'asst_TlTSxQoGlrUv1uAoBUWPwlhU', 'thread_id': 'thread_pHIzM7sGYEQAsLpN1xPiKJls', 'run_id': 'run_ZXuEfCu9bp2JpIjk6vdsQmGf', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The analysis has been completed, and the resulting chart is stored in the file `output/chart_1743095741.png`.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_osVEH9DLqMdmp9sHefFicdbr', 'object': 'thread.message', 'created_at': 1743095733, 'assistant_id': None, 'thread_id': 'thread_pHIzM7sGYEQAsLpN1xPiKJls', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Current date is 2025-03-27.model_deployment:gpt-4o, what were Profits for Carreterra in Germany in December?,output_folder:output, data_input:./data/financ

![alt text](output/example_output_chart.png)

In [14]:
print(response['value'])

The profits for Carreterra in Germany in December were 134,238.10.


---

## **7. Conclusion**

In this notebook, we've successfully transformed a script into an interactive Jupyter Notebook that sets up an AI agent using LIDA for data analysis. The agent can generate summaries, visualize data, and analyze generated charts, providing comprehensive insights based on user queries.

**Next Steps**:

- **Enhance Functionality**: Integrate additional tools or functionalities as needed.
- **Error Handling**: Implement more robust error handling for production environments.
- **User Interface**: Develop a user-friendly interface to interact with the AI agent seamlessly.
- **Deployment**: Deploy the AI agent to a cloud platform for wider accessibility.

Feel free to experiment with different data sources and questions to fully leverage the capabilities of your AI agent!

---